# Homework 02

# Table of Contents

* [first lambda](#first-lambda)
* [second lambda](#second-lambda)

In [20]:
import csv
with open("recalls-truncated.csv") as f:
    reader = csv.reader(f)
    headers = next(reader)

In [21]:
with open("recalls-truncated.csv") as f:
    reader = csv.reader(f)
    headers = next(reader)
    data = [(row[1], int(row[4])) for row in reader if row[3] == 'Vehicle']
data

[('Triple E Recreational Vehicles', 341),
 ('Volvo Car USA, LLC', 74),
 ('Volkswagen Group of America, Inc.', 1042),
 ('Indian Motorcycle Company', 4653),
 ('Lion Electric Company', 13),
 ('Mercedes-Benz USA, LLC', 323963),
 ('Ford Motor Company', 101001),
 ('Mercedes-Benz USA, LLC', 123696),
 ('Shadow Trailer, LLC', 12),
 ('Honda (American Honda Motor Co.)', 19),
 ('Chrysler (FCA US, LLC)', 38),
 ('Kibbi, LLC', 292),
 ('REV Recreation Group', 1540),
 ('Volkswagen Group of America, Inc.', 37558),
 ('BMW of North America, LLC', 14086),
 ('New Flyer of America, Inc.', 746),
 ('BMW of North America, LLC', 4),
 ('Mercedes-Benz USA, LLC', 11),
 ('Hyundai Motor America', 853),
 ('Lightning eMotors Inc.', 27),
 ('Mercedes-Benz USA, LLC', 1),
 ('Mercedes-Benz USA, LLC', 69),
 ('PACCAR Incorporated', 2565),
 ('Mercedes-Benz USA, LLC', 278),
 ('PACCAR Incorporated', 15658),
 ('Airstream, Inc.', 1992),
 ('General Motors, LLC', 111242),
 ('Airstream, Inc.', 200),
 ('General Motors, LLC', 9),
 ('Ja

## first lambda

In [22]:
highest = max(data, key=lambda x: x[1])
lowest = min(data, key=lambda x: x[1])

In [23]:
print(lowest, highest)

('Mercedes-Benz USA, LLC', 1) ('Chrysler (FCA US, LLC)', 1224078)


In [24]:
import nelta as nt

In [25]:
t = nt.read_csv('recalls-truncated.csv')
t.shape()

(350, 5)

In [26]:
t.columns

['Date', 'Manufacturer', 'Subject', 'Recall Type', 'Potentially Affected']

In [27]:
t.head(4)

             Date   Manufacturer Subject  Recall TypePotentially Affected
    0     01/06/2023Triple E Recreational VehiclesBattery Disconnect Switch May Short Vehicle    341.0   
    1     01/05/2023Volvo Car USA, LLCSteering Wheel May Lock Up Vehicle     74.0   
    2     12/29/2022Volkswagen Group of America, Inc.12-Volt Battery Cable May Short Circuit Vehicle    1042.0  
    3     12/29/2022Indian Motorcycle CompanyKickstand May Not Retract Properly/FMVSS 123 Vehicle    4653.0  

In [28]:
last_four = t.tail(4)
last_four

             Date   Manufacturer Subject  Recall TypePotentially Affected
   346    08/25/2022Rivian Automotive, LLCImproperly Secured Front Seat Belt Anchor Vehicle    207.0   
   347    08/25/2022Chrysler (FCA US, LLC)Rearview Camera Image May Not Display/FMVSS 111 Vehicle    7895.0  
   348    08/25/2022General Motors, LLCSecondary Hood Latch Corrosion Vehicle   120688.0 
   349    08/25/2022PT. Elangperdana Tyre IndustryTire Sidewall Separation/ FMVSS 139   Tire    31500.0  

In [29]:
ll = nt.LabeledList(last_four['Subject'], [346 + i for i in range(4)])
ll

346       Improperly Secured Front Seat Belt Anchor
347 Rearview Camera Image May Not Display/FMVSS 111
348                  Secondary Hood Latch Corrosion
349             Tire Sidewall Separation/ FMVSS 139

In [30]:
for val in ll:
    print(val)

Improperly Secured Front Seat Belt Anchor
Rearview Camera Image May Not Display/FMVSS 111
Secondary Hood Latch Corrosion
Tire Sidewall Separation/ FMVSS 139


In [31]:
last_four[['Manufacturer', 'Subject']]

          Manufacturer Subject  
   346    Rivian Automotive, LLCImproperly Secured Front Seat Belt Anchor
   347    Chrysler (FCA US, LLC)Rearview Camera Image May Not Display/FMVSS 111
   348    General Motors, LLCSecondary Hood Latch Corrosion
   349    PT. Elangperdana Tyre IndustryTire Sidewall Separation/ FMVSS 139

## second lambda

In [32]:
vehicle = t['Recall Type']
vehicle = list(map(lambda x: x == 'Vehicle', vehicle))
vehicles = t[vehicle]
len(vehicles.index)

313

In [33]:
my_filter = vehicles['Potentially Affected']>500000
my_filter

  0 False
  1 False
  2 False
  3 False
  5 False
  7 False
  8 False
  9 False
 10 False
 11 False
 12 False
 13 False
 14 False
 15 False
 16 False
 17 False
 18 False
 19 False
 20 False
 21 False
 22 False
 23 False
 24 False
 25 False
 26 False
 27 False
 28 False
 30 False
 31 False
 33 False
 34 False
 35 False
 36 False
 37 False
 38 False
 40 False
 41 False
 42 False
 43 False
 44 False
 45 False
 46 False
 47 False
 48 False
 49 False
 50 False
 51 False
 52 False
 53 False
 54 False
 55 False
 56 False
 57 False
 58 False
 60  True
 61  True
 62 False
 63 False
 64 False
 65 False
 66 False
 67 False
 68 False
 69 False
 71 False
 73 False
 74 False
 75 False
 76 False
 77 False
 78 False
 79 False
 80 False
 81 False
 82 False
 83 False
 84 False
 85 False
 86 False
 89 False
 90 False
 91 False
 92 False
 93 False
 94 False
 95 False
 96 False
 97 False
 98 False
 99 False
100 False
101 False
102 False
103 False
104 False
105 False
106 False
107 False
108 False
109 False


In [34]:
type(my_filter)

nelta.LabeledList

In [35]:
my_filter[3]

False

In [36]:
rare_occurence = vehicles[list(my_filter)]
rare_occurence

             Date   Manufacturer Subject  Recall TypePotentially Affected
    60    12/08/2022General Motors, LLCRunning Lights May Not Deactivate/FMVSS 108 Vehicle   740108.0 
    61    12/08/2022Chrysler (FCA US, LLC)Tailgate May Open While Driving Vehicle  1224078.0 
   110    11/18/2022Ford Motor CompanyCracked Fuel Injector May Leak and Cause a Fire Vehicle   521746.0 
   284    09/19/2022Tesla, Inc.Power Windows May Pinch/FMVSS 118 Vehicle  1096762.0 

In [37]:
manufacturer = rare_occurence['Manufacturer']
uppercase_manufacturer = manufacturer.map(lambda x: x.upper())
uppercase_manufacturer

 60    GENERAL MOTORS, LLC
 61 CHRYSLER (FCA US, LLC)
110     FORD MOTOR COMPANY
284            TESLA, INC.

In [38]:
t[list(map(lambda x: x.split('/')[2] == '2023', t['Date']))]

             Date   Manufacturer Subject  Recall TypePotentially Affected
    0     01/06/2023Triple E Recreational VehiclesBattery Disconnect Switch May Short Vehicle    341.0   
    1     01/05/2023Volvo Car USA, LLCSteering Wheel May Lock Up Vehicle     74.0   